In [ ]:
from haystack.utils import launch_es
launch_es()

In [ ]:
from qasystem.QASystem import *
from evaluation.xquadevaluation import *
from tqdm.contrib import tzip

In [ ]:
db = Database()
db.connect()
db.delete_all_documents()
# df, dicts = xquad_data_prepare('/home/doinakis/github/haystack/xquad-dataset/xquad.el.json')
# db.add_documents(dicts=dicts)
# qa = QASystem(database=db)


In [ ]:
from scrapping.CNNScrape import *

In [ ]:
docs = scrape_init("./test_db/politiki", "politiki", 0, 2)

In [ ]:
db.add_documents(docs)

In [ ]:
f1_scores = []
em_scores = []
retriever_em = []
actual_ans = []
predicted_ans = []
for question, answer, doc_id in tzip(df.question, df.answer, df.doc_id):
  prediction = qa.pipe.run(
    query=f"{question}", params={"ESRetriever": {"top_k": 3}, "Reader": {"top_k": 1}}
  )
  actual_ans.append(answer['text'])
  predicted_ans.append(prediction['answers'][0].answer)
  retriever_em.append(int(prediction['documents'][0].meta['name'] == doc_id))
  em = compute_em(prediction['answers'][0].answer, answer['text'])
  em_scores.append(em)
  if em == 1:
    f1_scores.append(1)
  else:
    f1_scores.append(compute_f1(prediction['answers'][0].answer, answer['text']))

scores = pd.DataFrame()
scores['actual'] = actual_ans
scores['predicted'] =predicted_ans
scores['em'] = em_scores
scores['f1'] = f1_scores
scores['retriever_em'] = retriever_em

print(f'Exact Match: {scores.em.mean()}')
print(f'F1-Score: {scores.f1.mean()}')
print(f'Accuracy: {scores.retriever_em.mean()}')

In [ ]:
import pandas as pd

In [ ]:
topic_data = pd.read_json('topic_data.json')

In [ ]:
for index, data in topic_data.iterrows():
  data.label = ''.join(data.label)

In [ ]:
to_keep = (topic_data.label == 'Τηλεόραση') | \
  (topic_data.label == 'Τέχνες-Πολιτισμός') | \
  (topic_data.label == 'Επιστολές')         | \
  (topic_data.label == 'Αστυνομικά')        | \
  (topic_data.label == 'Ζώδια')             | \
  (topic_data.label == 'Καιρός')            | \
  (topic_data.label == 'Διαφημίσεις')       | \
  (topic_data.label == 'Βιογραφίες')        | \
  (topic_data.label == 'Κοινωνικά')         | \
  (topic_data.label == 'Συνεντεύξεις')      | \
  (topic_data.label == 'Διάφορα')

In [ ]:
new_topic = topic_data.drop(topic_data[~to_keep].index).reset_index()

In [ ]:
new_topic.label.value_counts()

In [ ]:
other_data_df = pd.DataFrame()
other_data_df['content'] = new_topic.text
other_data_df['url'] = 'no_url'
other_data_df['name'] = 'makedonia'
other_data_df['title'] = 'no_title'
other_data_df['category'] = 'other'

In [ ]:
kappa =  pd.read_json('/home/doinakis/github/Real-Time-News-Assistant/db/train_dataset.json')

In [ ]:
kappa.__len__()

In [ ]:
kappa.shape[0]

In [ ]:
date = "2022-05-01T00:00:00.000+02:00"

In [ ]:
date = date.split('T')[0]

In [ ]:
date

In [ ]:
from datetime import datetime

In [ ]:
then = datetime.strptime(date, '%Y-%m-%d')

In [ ]:
now = datetime.now()

In [ ]:
now.time()

In [ ]:
now > then

In [ ]:
from haystack.nodes import BM25Retriever
from qasystem.QASystem import *
db = Database()
db.connect()

In [ ]:
bm25 = BM25Retriever(db.document_store)

In [ ]:
documents= bm25.retrieve(query="ποσους ποντους παρεδωσε η αμυνα?", top_k=2)

In [ ]:
documents[0].meta

In [ ]:
for document in documents:
  print(document.meta['name'])

In [ ]:
names = []
for document in documents:
  names.append(document.meta['name'])

In [ ]:
names

In [ ]:
int('document0.jdsson' in names)